In [2]:
import pandas as pd
import tensorflow as tf
import sklearn


In [4]:
pd.__version__, tf.__version__, sklearn.__version__

('2.0.3', '2.16.1', '1.5.0')

In [5]:
from tensorflow.keras.layers import Dense, Dropout, Activation, Input
from tensorflow.keras.models import Model
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [9]:
base = pd.read_csv('games.csv')

In [10]:
base = base.drop('Other_Sales', axis = 1)
base = base.drop('Global_Sales', axis = 1)
base = base.drop('Developer', axis = 1)

In [11]:
base.shape

(16719, 13)

In [12]:
base.isnull().sum()

Name                  2
Platform              0
Year_of_Release     269
Genre                 2
Publisher            54
NA_Sales              0
EU_Sales              0
JP_Sales              0
Critic_Score       8582
Critic_Count       8582
User_Score         6704
User_Count         9129
Rating             6769
dtype: int64

In [13]:
base = base.dropna(axis = 0)

In [14]:
base.shape

(6825, 13)

In [15]:
base.isnull().sum()

Name               0
Platform           0
Year_of_Release    0
Genre              0
Publisher          0
NA_Sales           0
EU_Sales           0
JP_Sales           0
Critic_Score       0
Critic_Count       0
User_Score         0
User_Count         0
Rating             0
dtype: int64

In [16]:
base ['Name']. value_counts()

Name
Need for Speed: Most Wanted                  8
Madden NFL 07                                8
LEGO Star Wars II: The Original Trilogy      8
The Sims 2                                   7
Terraria                                     7
                                            ..
Castlevania: Portrait of Ruin                1
Suzuki TT Superbikes                         1
Rumble Roses                                 1
Sherlock Holmes: The Mystery of the Mummy    1
STORM: Frontline Nation                      1
Name: count, Length: 4377, dtype: int64

In [17]:
base = base.drop('Name', axis = 1)

In [18]:
base.shape

(6825, 12)

In [22]:
X = base.iloc[:, [0, 1, 2, 3, 7, 8, 9, 10, 11]].values

In [23]:
y_na = base.iloc[:,4].values
y_eu = base.iloc[:,5].values
y_jp = base.iloc[:,6].values

In [24]:
y_na

array([4.136e+01, 1.568e+01, 1.561e+01, ..., 0.000e+00, 1.000e-02,
       0.000e+00])

In [25]:
y_eu

array([2.896e+01, 1.276e+01, 1.093e+01, ..., 1.000e-02, 0.000e+00,
       1.000e-02])

In [26]:
y_jp

array([3.77, 3.79, 3.28, ..., 0.  , 0.  , 0.  ])

In [28]:
base['Platform'].value_counts()

Platform
PS2     1140
X360     858
PS3      769
PC       651
XB       565
Wii      479
DS       464
PSP      390
GC       348
PS4      239
GBA      237
XOne     159
3DS      155
PS       150
PSV      118
WiiU      89
DC        14
Name: count, dtype: int64

In [30]:
onehotencoder = ColumnTransformer(transformers=[('OneHot', OneHotEncoder(), [0, 2, 3, 8])], remainder = 'passthrough')
X = onehotencoder.fit_transform(X).toarray()

In [33]:
(303 + 3)/ 2

153.0

In [34]:
camada_entrada = Input(shape = (303,))
camada_oculta1 = Dense(units = 153, activation = 'relu')(camada_entrada)
camada_oculta2 = Dense(units = 153, activation = 'relu')(camada_oculta1)
camada_saida1 = Dense(units = 1, activation = 'linear')(camada_oculta2)
camada_saida2 = Dense(units = 1, activation = 'linear')(camada_oculta2)
camada_saida3 = Dense(units = 1, activation = 'linear')(camada_oculta2)

In [35]:
regressor = Model(inputs = camada_entrada, outputs = [camada_saida1, camada_saida2, camada_saida3])

In [37]:
regressor.compile(optimizer='adam', loss = 'mse')

epochs = 500

In [38]:
regressor.fit(X, [y_na, y_eu, y_jp], epochs = 100, batch_size = 100)

Epoch 1/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - dense_2_loss: 1533.9626 - dense_3_loss: 783.6218 - dense_4_loss: 2023.2482 - loss: 4341.2935  
Epoch 2/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - dense_2_loss: 2.2611 - dense_3_loss: 2.1944 - dense_4_loss: 2.6257 - loss: 7.0822
Epoch 3/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - dense_2_loss: 1.4965 - dense_3_loss: 1.0512 - dense_4_loss: 1.0851 - loss: 3.6328
Epoch 4/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - dense_2_loss: 1.8210 - dense_3_loss: 1.1903 - dense_4_loss: 0.8149 - loss: 3.8267
Epoch 5/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - dense_2_loss: 0.9595 - dense_3_loss: 0.6097 - dense_4_loss: 0.8608 - loss: 2.4305
Epoch 6/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - dense_2_loss: 0.8118 - dense_3_loss: 0.5142 - dense_4_loss: 0.8862 - loss: 2.2126
Epoch 7/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - dense_2_loss: 0.7899 - dense_3_loss: 0.5150 - dense_4_loss: 0.8359 - loss: 2.1397
Epoch 8/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 

In [40]:
previsao_na, previsao_eu, previsao_jp = regressor.predict(X)

214/214 ━━━━━━━━━━━━━━━━━━━━ 0s 860us/step


In [43]:
previsao_na, previsao_na.mean()

(array([[ 2.5779917 ],
        [ 3.235518  ],
        [ 2.489273  ],
        ...,
        [ 0.34672663],
        [ 0.01703736],
        [-0.1506181 ]], dtype=float32),
 0.7869846)

In [44]:
y_na, y_na.mean()

(array([4.136e+01, 1.568e+01, 1.561e+01, ..., 0.000e+00, 1.000e-02,
        0.000e+00]),
 0.3944835164835165)

In [45]:
from sklearn.metrics import mean_absolute_error

In [46]:
mean_absolute_error(y_na, previsao_na)

0.57022112112277

In [47]:
previsao_eu, previsao_eu.mean()

(array([[ 0.72520304],
        [ 1.425036  ],
        [ 0.5740094 ],
        ...,
        [-0.8408427 ],
        [-0.7326796 ],
        [-0.7034843 ]], dtype=float32),
 -0.18109329)

In [48]:
y_eu, y_eu.mean()

(array([2.896e+01, 1.276e+01, 1.093e+01, ..., 1.000e-02, 0.000e+00,
        1.000e-02]),
 0.23608937728937732)

In [51]:
mean_absolute_error(y_eu, previsao_eu)

0.4476002404696434

In [49]:
previsao_jp, previsao_jp.mean()

(array([[-0.7003378 ],
        [-1.0341038 ],
        [-0.64904827],
        ...,
        [-1.3888568 ],
        [-1.6025361 ],
        [-1.6417903 ]], dtype=float32),
 -1.4237758)

In [50]:
y_jp, y_jp.mean()

(array([3.77, 3.79, 3.28, ..., 0.  , 0.  , 0.  ]), 0.06415824175824175)

In [52]:
mean_absolute_error(y_jp, previsao_jp)

1.4879340139922643